# Calculating 5-minute moving average for each stock symbol

In [0]:
from pyspark.sql import functions as F

# Read from Silver table
silver_df = df = spark.read.table("kafka.silver.silver_stock_data")

# Example of calculating 5-minute moving average for each stock symbol
gold_df = (silver_df
           .groupBy(F.window("timestamp", "5 minutes"), "symbol")
           .agg(F.avg("high").alias("avg_price")))
           # Calculate 50-period simple moving average for each stock symbol
ohlc_df = silver_df.groupBy(F.window("timestamp", "1 day"), "symbol").agg(
    F.first("open").alias("open_price"),
    F.max("high").alias("high_price"),
    F.min("low").alias("low_price"),
    F.last("close").alias("close_price"),
    F.sum("volume").alias("daily_volume")
)

# Write to the Gold Delta table
#gold_df.write.format("delta").mode("overwrite").saveAsTable("kafka.gold.gold_stock_data")

ohlc_df.write.format("delta").mode("overwrite") \
.option("mergeSchema", "true") \
.saveAsTable("kafka.gold.gold_stock_data") 



In [0]:
%sql
select * from kafka.gold.gold_stock_data

window,symbol,avg_price,open_price,high_price,low_price,close_price,daily_volume
"List(2024-10-04T00:00:00Z, 2024-10-05T00:00:00Z)",MSFT,null,415.8800,418.1800,415.6025,415.6550,115084.0
"List(2024-10-04T00:00:00Z, 2024-10-05T00:00:00Z)",GOOGL,null,166.9500,167.0600,164.5400,166.8500,579984.0
"List(2024-10-04T00:00:00Z, 2024-10-05T00:00:00Z)",AAPL,null,226.3700,226.8000,224.4200,226.6350,950900.0


#-- Daily stock price aggregates

In [0]:
%sql
-- Daily stock price aggregates
CREATE OR REPLACE TABLE kafka.gold.daily_stock_aggregates AS
SELECT 
  symbol,
  timestamp,
  AVG(open) AS avg_open,
  AVG(close) AS avg_close,
  MAX(high) AS max_high,
  MIN(low) AS min_low,
  SUM(volume) AS total_volume
FROM kafka.silver.silver_stock_data
GROUP BY symbol, timestamp;


num_affected_rows,num_inserted_rows


# Daily stock aggregation

In [0]:
%sql
select * from kafka.gold.daily_stock_aggregates order by symbol

symbol,timestamp,avg_open,avg_close,max_high,min_low,total_volume
AAPL,2024-10-04 19:48:00,226.43,226.48500000000007,226.4900,226.4200,1530.0
AAPL,2024-10-04 18:59:00,226.565,226.565,226.7000,226.5600,1290.0
AAPL,2024-10-04 18:55:00,226.54000000000002,226.53000000000003,226.5400,226.5300,550.0
AAPL,2024-10-04 18:39:00,226.53500000000003,226.53000000000003,226.6300,226.5300,7990.0
AAPL,2024-10-04 19:00:00,226.45,226.575,226.5750,226.4500,60.0
AAPL,2024-10-04 18:38:00,226.53500000000003,226.53500000000003,226.5400,226.5300,270.0
AAPL,2024-10-04 19:04:00,226.575,226.45,226.5750,226.4500,260.0
AAPL,2024-10-04 19:16:00,226.555,226.5,226.5550,226.5000,280.0
AAPL,2024-10-04 19:58:00,225.29000000000002,226.41890000000006,226.4600,225.2900,1450.0
AAPL,2024-10-04 19:38:00,226.45999999999998,226.45999999999998,226.4600,226.4600,360.0


In [0]:
%sql
SELECT * FROM kafka.gold.gold_stock_data ;

window,symbol,avg_price,open_price,high_price,low_price,close_price,daily_volume
"List(2024-10-04T00:00:00Z, 2024-10-05T00:00:00Z)",MSFT,null,415.8800,418.1800,415.6025,415.6550,115084.0
"List(2024-10-04T00:00:00Z, 2024-10-05T00:00:00Z)",GOOGL,null,166.9500,167.0600,164.5400,166.8500,579984.0
"List(2024-10-04T00:00:00Z, 2024-10-05T00:00:00Z)",AAPL,null,226.3700,226.8000,224.4200,226.6350,950900.0


In [0]:
from pyspark.sql.functions import max as spark_max, min as spark_min, sum as spark_sum

# Monthly Aggregates
monthly_aggregates = silver_df.groupBy("symbol", "timestamp").agg(
    spark_max("high").alias("max_high"),
    spark_min("low").alias("min_low"),
    spark_sum("volume").alias("total_volume")
)

# Write the monthly aggregates to Delta table
monthly_aggregates.write.format("delta").mode("overwrite").saveAsTable("kafka.gold.monthly_aggregates")

In [0]:
%sql
select * from kafka.gold.monthly_aggregates order by symbol

symbol,timestamp,max_high,min_low,total_volume
AAPL,2024-10-04 19:48:00,226.4900,226.4200,1530.0
AAPL,2024-10-04 18:59:00,226.7000,226.5600,1290.0
AAPL,2024-10-04 18:55:00,226.5400,226.5300,550.0
AAPL,2024-10-04 18:39:00,226.6300,226.5300,7990.0
AAPL,2024-10-04 19:00:00,226.5750,226.4500,60.0
AAPL,2024-10-04 18:38:00,226.5400,226.5300,270.0
AAPL,2024-10-04 19:04:00,226.5750,226.4500,260.0
AAPL,2024-10-04 19:16:00,226.5550,226.5000,280.0
AAPL,2024-10-04 19:58:00,226.4600,225.2900,1450.0
AAPL,2024-10-04 19:38:00,226.4600,226.4600,360.0


# Optimize Delta Tables

In [0]:
%sql
OPTIMIZE kafka.gold.daily_stock_aggregates ZORDER BY (symbol)


path,metrics
abfss://data@adbmathew01.dfs.core.windows.net/ext_tables/gold/__unitystorage/schemas/1b6382ee-4640-4e7b-bc3c-4bf9b5517b79/tables/6966fa29-10e5-44d2-9036-92ae8b369be9,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 8051), 0, List(0, 0), 0, null), null, 0, 0, 1, 1, false, 0, 0, 1728317014422, 1728317015709, 4, 0, null, List(0, 0), 7, 7, 0, 0, null)"


In [0]:
%sql
SELECT * FROM kafka.gold.gold_stock_data ;

window,symbol,avg_price,open_price,high_price,low_price,close_price,daily_volume
"List(2024-10-04T00:00:00Z, 2024-10-05T00:00:00Z)",MSFT,null,415.8800,418.1800,415.6025,415.6550,115084.0
"List(2024-10-04T00:00:00Z, 2024-10-05T00:00:00Z)",GOOGL,null,166.9500,167.0600,164.5400,166.8500,579984.0
"List(2024-10-04T00:00:00Z, 2024-10-05T00:00:00Z)",AAPL,null,226.3700,226.8000,224.4200,226.6350,665630.0


In [0]:
%sql
select * from kafka.gold.daily_stock_aggregates

symbol,timestamp,avg_open,avg_close,max_high,min_low,total_volume
MSFT,2024-10-04 18:45:00,415.95,415.84,415.9500,415.8000,305.0
GOOGL,2024-10-04 18:33:00,166.86000000000004,166.86999999999998,166.8700,166.8600,552.0
AAPL,2024-10-04 19:58:00,225.29,226.41890000000006,226.4600,225.2900,1015.0
GOOGL,2024-10-04 19:57:00,166.93000000000004,166.98,166.9800,166.8800,5736.0
GOOGL,2024-10-04 19:51:00,166.87799999999996,166.80400000000003,166.8800,166.8040,264.0
GOOGL,2024-10-04 19:00:00,166.86999999999998,166.88000000000002,166.8800,166.8700,60.0
AAPL,2024-10-04 18:31:00,226.59999999999997,226.58999999999997,226.6400,226.5400,665.0
MSFT,2024-10-04 19:38:00,415.84999999999997,415.84,415.8500,415.8300,1206.0
MSFT,2024-10-04 19:53:00,415.88000000000005,415.88000000000005,415.8800,415.8250,258.0
MSFT,2024-10-04 18:32:00,415.7,415.68999999999994,415.7000,415.6100,1925.0
